In [9]:
import sys

sys.path.insert(0, '/home/jianx/search-exposure/')
import faiss
import forward_ranker.load_data as load_data
from forward_ranker.utils import print_message
import numpy as np
from forward_ranker.test import get_ndcg_precision_rr
obj_reader = load_data.obj_reader
obj_writer = load_data.obj_writer

In [5]:
print_message("Loading embeddings.")
query_test_emb = obj_reader("/home/jianx/results/test_query_0__emb_p__data_obj_0.pb")
qid_test_offset = obj_reader("/datadrive/data/preprocessed_data_with_test/test-query_qid2offset.pickle")
passage_embeddings = obj_reader("/home/jianx/results/passage_0__emb_p__data_obj_0.pb")
pid_mapping = obj_reader("/datadrive/jianx/data/annoy/100_ance_passage_map.dict")
initial_ranking = obj_reader("/datadrive/jianx/data/initial_ranking.dict")
pid_offset = obj_reader("/datadrive/data/preprocessed_data_with_test/pid2offset.pickle")

[Sep 09, 00:55:09] Loading embeddings.


In [6]:
print(initial_ranking)

{494835: [7130104, 7132810, 8004107, 7135097, 8027344, 80373, 8464186, 5210824, 8063787, 8477675, 8073459, 8479962, 722764, 8074471, 8497424, 8111413, 7292392, 5298846, 7324621, 531040, 7339432, 8183988, 7361462, 8186068, 8605525, 8198790, 7381566, 8213010, 8213011, 5377419, 7427810, 8658703, 5408609, 8266165, 7457368, 7459313, 7468404, 7468408, 544110, 747832, 547424, 5478394, 7516506, 8318151, 5486463, 7525588, 7530941, 550290, 8339118, 837341, 8375395, 8381938, 8814730, 8392522, 8395043, 6311334, 6317997, 903055, 6350715, 6001941, 93388, 6027063, 6039804, 97739, 6405074, 1511116, 1512676, 6092171, 6125590, 6145377, 6496889, 6500324, 6502706, 6161869, 6520291, 617668, 6187402, 1621580, 6226492, 6227476, 6592777, 1686441, 6627579, 1686444, 6660619, 1716161, 667608, 669141, 2207775, 2211254, 3069582, 3074713, 3080669, 3072598, 2246320, 226354, 2279956, 3139495, 3146622, 3152904, 3159911, 3165207, 3174178, 2333748, 322092, 3219547, 2403974, 3253160, 3257190, 3262059, 3281460, 3276049, 3

In [12]:
qid_test_mapping = {v: k for k, v in qid_test_offset.items()}
result_dict = {}
dim = passage_embeddings.shape[1]


for idx, q_emb in enumerate(query_test_emb):
    qid = qid_test_mapping[idx]
    index = faiss.IndexFlatIP(dim)
    p_indices = []
    for i in initial_ranking[qid]:
        if i in pid_offset:
            p_indices.append(pid_offset[i])
    p_embs = passage_embeddings[np.array(p_indices)]
    index.add(p_embs)
    _, top_list = index.search(np.array([q_emb]), 1000)
    result_dict[qid] = {}
    for j, k in enumerate(top_list[0]):
        result_dict[qid][initial_ranking[qid][k]] = 1000 - j

In [13]:
import random
import math
rating_dict = obj_reader("/datadrive/jianx/data/rel_scores.dict")
rank = 10

qids = list(result_dict.keys())
result_ndcg = []
result_prec = []
result_rr = []
for qid in qids:
    if qid in rating_dict:
        ndcg, prec, rr = get_ndcg_precision_rr(rating_dict[qid], result_dict[qid], rank)
        result_ndcg.append(ndcg)
        result_prec.append(prec)
        result_rr.append(rr)
avg_ndcg = np.nanmean(result_ndcg)
avg_prec = np.nanmean(result_prec)
avg_rr = np.nanmean(result_rr)
print(avg_ndcg, avg_prec, avg_rr)

0.6620657697757945 0.7441860465116278 0.9348837209302326


In [ ]:
# Generate Negative Data
with open("/datadrive/jianx/data/results/rerank_search_rankings_100_100_flat.csv", "r") as f:
    for line in f:
        split = line.split()